In [1]:
!pip -q install gradio plotly


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 9.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.1 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


# 🌍 CO₂ Scenario Simulator — Versi Publik

### 💡 Apa itu simulasi?
Simulasi ini digunakan untuk *menguji berbagai kebijakan iklim secara sederhana* — misalnya:
- Mengurangi deforestasi (LUC)
- Mengganti pembangkit batubara dengan energi bersih
- Meningkatkan efisiensi energi di rumah dan industri

Dengan menggeser slider, kamu bisa melihat bagaimana total emisi CO₂ suatu negara akan berubah jika kebijakan tersebut diterapkan.

---

### 📆 Apa itu baseline?
**Baseline** adalah titik awal perbandingan — seperti “kondisi awal” sebelum kebijakan diterapkan.  — bisa berupa:
- **Single-year baseline**: menggunakan data pada satu tahun tertentu (biasanya tahun terakhir yang datanya lengkap).  
  → Cocok untuk visualisasi dan demo cepat.
- **Multi-year baseline (3-year median)**: menghitung rata-rata tengah dari 3 tahun terakhir.  
  → Cocok untuk analisis lebih stabil karena tidak terpengaruh tahun ekstrem (misalnya 2015 kebakaran besar atau 2020 pandemi).

---

### 🔥 Apa itu *flaring*?
**Flaring** adalah proses **pembakaran gas buangan** (biasanya metana) di industri minyak dan gas.  
Ini dilakukan untuk alasan keamanan atau teknis, tetapi menghasilkan **CO₂ langsung ke atmosfer**.  

📘 *Contoh:*  
> Di kilang minyak atau ladang gas, jika tekanan gas berlebih, gas tersebut dibakar lewat cerobong tinggi (*flare stack*) untuk mencegah ledakan — proses inilah yang disebut *flaring*.

Mengurangi flaring berarti **mengurangi emisi gas rumah kaca langsung** dari sektor energi.

---

### 🎯 Tujuan simulasi
Tujuannya bukan untuk menghasilkan angka pasti, tetapi untuk **memahami arah perubahan**:
> “Jika deforestasi turun 30%, seberapa besar emisi nasional bisa berkurang?”

Hasilnya bisa membantu publik, jurnalis, dan pengambil kebijakan memahami efek keputusan lingkungan secara cepat dan visual.

---

### ⚙️ Cara Menggunakan
1. Pilih negara dan mode baseline (1 tahun / 3 tahun median).  
2. Atur pengurangan tiap sektor atau tingkat efisiensi energi.  
3. Klik **“Jalankan Simulasi”** untuk melihat perubahan total emisi dan grafiknya.  
4. (Opsional) Atur target tahun untuk proyeksi linear (misalnya 2030).


In [16]:
import os, numpy as np, pandas as pd
import plotly.express as px
import gradio as gr

# =======================
# Konfigurasi & Kolom
# =======================
DATA_DEFAULT = "/kaggle/input/asaaaaaaaa/co2_country_clean.csv"
FORECAST_DEFAULT = "/kaggle/input/csv-forecast/forecast_full_indonesia_to_2035.csv"
SECTOR_COLS_ALL = ["coal_co2","oil_co2","gas_co2","cement_co2","flaring_co2","land_use_change_co2"]
TARGET_MAX_YEAR = 2035
COUNTRY = "Indonesia"

# =======================
# Loader & Helpers
# =======================
def load_data(default_path=DATA_DEFAULT):
    df = pd.read_csv(default_path)
    if "description" in df.columns:
        df = df[df["description"].str.lower()=="country"].copy()
    df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
    return df

def latest_clean_year(df, country, sector_cols):
    d = df[df["name"]==country].copy()
    have = [c for c in sector_cols if c in d.columns]
    if not have: return int(d["year"].max())
    d["total"] = d[have].fillna(0).sum(axis=1)
    nonzero = d[d["total"]>0]
    return int(nonzero["year"].max()) if not nonzero.empty else int(d["year"].max())

def load_forecast_series(csv_path):
    try:
        df = pd.read_csv(csv_path)
        if "predicted_mtco2" not in df.columns:
            for c in ["total","yhat","forecast","value"]:
                if c in df.columns:
                    df.rename(columns={c:"predicted_mtco2"}, inplace=True)
                    break
        df = df[["year","predicted_mtco2"]].dropna()
        df["year"] = pd.to_numeric(df["year"], errors="coerce").astype(int)
        return df[df["year"]<=TARGET_MAX_YEAR]
    except Exception:
        return pd.DataFrame(columns=["year","predicted_mtco2"])

def _kfmt(v):
    return "NA" if pd.isna(v) else f"{v:,.2f}"

# =======================
# Core simulate
# =======================
def simulate(cut_luc, cut_coal, cut_oil, cut_gas, cut_cement, cut_flaring,
             eff_improve, target_year):

    df = load_data()
    fc_df = load_forecast_series(FORECAST_DEFAULT)
    sector_cols = [c for c in SECTOR_COLS_ALL if c in df.columns]

    # baseline otomatis Indonesia (tahun terakhir bersih)
    year_used = latest_clean_year(df, COUNTRY, sector_cols)
    d = df[df["name"]==COUNTRY]
    base = d[d["year"]==year_used]
    baseline = {c: float(base[c].iloc[0]) if c in base.columns else 0.0 for c in sector_cols}
    baseline_total = sum(baseline.values())

    # Apply reductions
    cuts = {
        "land_use_change_co2": cut_luc,
        "coal_co2": cut_coal,
        "oil_co2": cut_oil,
        "gas_co2": cut_gas,
        "cement_co2": cut_cement,
        "flaring_co2": cut_flaring,
    }
    sim = baseline.copy()
    for k, pct in cuts.items():
        if k in sim:
            sim[k] *= (1 - pct/100.0)

    # Efisiensi energi
    energy_keys = [k for k in ["coal_co2","oil_co2","gas_co2","cement_co2","flaring_co2"] if k in sim]
    for k in energy_keys:
        sim[k] *= (1 - eff_improve/100.0)

    sim_total = sum(sim.values())
    delta_abs = sim_total - baseline_total
    delta_pct = (delta_abs / baseline_total * 100.0) if baseline_total > 0 else np.nan

    # ----- Markdown Ringkasan -----
    md = f"""
### 📌 Ringkasan Skenario — {COUNTRY}
Baseline otomatis: {year_used}

- **Total baseline:** {_kfmt(baseline_total)} MtCO₂  
- **Total simulasi:** {_kfmt(sim_total)} MtCO₂  
- **Perubahan:** {_kfmt(delta_abs)} MtCO₂ (**{delta_pct:+.2f}%**)  

**Catatan:**  
- Efisiensi energi menurunkan sektor energi (coal/oil/gas/cement/flaring) secara merata.  
- Target tahun proyeksi dibatasi 2024–2035.
""".strip()

    # ----- Bar chart -----
    plot_df = pd.DataFrame({
        "Sektor": [s.replace("_co2","").upper() for s in sector_cols],
        "Baseline": [baseline.get(s, 0.0) for s in sector_cols],
        "Simulasi": [sim.get(s, 0.0) for s in sector_cols],
    })
    melt = plot_df.melt(id_vars="Sektor", value_vars=["Baseline","Simulasi"],
                        var_name="Kondisi", value_name="Emisi (MtCO₂)")
    fig_bar = px.bar(melt, x="Sektor", y="Emisi (MtCO₂)", color="Kondisi",
                     barmode="group", title=f"Perbandingan Emisi per Sektor — {COUNTRY}")

    # ----- Line forecast -----
    if fc_df.empty:
        years = [year_used, target_year]
        vals = [baseline_total, sim_total]
        proj_total = pd.DataFrame({"year": years, "total": vals})
    else:
        fc_df = fc_df.copy()
        fc_df = fc_df[(fc_df["year"] >= year_used) & (fc_df["year"] <= target_year)]
        if not fc_df.empty:
            ref_target = fc_df["predicted_mtco2"].iloc[-1]
            scale = (sim_total / ref_target) if ref_target > 0 else 1.0
            fc_df["total"] = fc_df["predicted_mtco2"] * scale
            proj_total = fc_df[["year","total"]]
        else:
            proj_total = pd.DataFrame({"year":[year_used,target_year],"total":[baseline_total,sim_total]})
    fig_line = px.line(proj_total, x="year", y="total", markers=True,
                       title=f"Total Emisi {COUNTRY} — Proyeksi Hingga {target_year}")

    # ----- Table + CSV -----
    col_order = ["sector", "baseline_mtco2", "simulated_mtco2", "percent_change(%)", "projection_year"]
    rows = []
    for s in sector_cols:
        base_v = float(baseline.get(s, 0.0))
        sim_v  = float(sim.get(s, 0.0))
        pct    = ((sim_v - base_v) / base_v * 100.0) if base_v > 0 else np.nan
        rows.append({
            "sector": s.replace("_co2","").upper(),
            "baseline_mtco2": base_v,
            "simulated_mtco2": sim_v,
            "percent_change(%)": pct,
            "projection_year": target_year
        })
    total_pct = ((sim_total - baseline_total) / baseline_total * 100.0)
    rows.append({
        "sector": "TOTAL",
        "baseline_mtco2": baseline_total,
        "simulated_mtco2": sim_total,
        "percent_change(%)": total_pct,
        "projection_year": target_year
    })
    res_df = pd.DataFrame(rows, columns=col_order)

    csv_path = f"/kaggle/working/scenario_{COUNTRY}_{year_used}_to_{target_year}.csv"
    res_df.to_csv(csv_path, index=False)

    md_year = f"**Tahun proyeksi: {target_year}** (dibatasi 2024–2035)"
    return md, fig_bar, fig_line, res_df, csv_path, md_year

# =======================
# UI Gradio
# =======================
with gr.Blocks(title="CO₂ Scenario Simulator (Indonesia only)") as demo:
    gr.Markdown("# 🌍 CO₂ Scenario Simulator — Indonesia\nUji skenario pengurangan emisi hingga tahun 2035.")

    # Skenario sliders
    with gr.Row():
        cut_luc = gr.Slider(0, 100, 0, 1, label="🌳 LUC (deforestasi & lahan)")
        cut_coal = gr.Slider(0, 100, 0, 1, label="🪨 Batubara")
        cut_oil = gr.Slider(0, 100, 0, 1, label="🛢️ Minyak")
    with gr.Row():
        cut_gas = gr.Slider(0, 100, 0, 1, label="🔥 Gas")
        cut_cement = gr.Slider(0, 100, 0, 1, label="🏗️ Semen")
        cut_flaring = gr.Slider(0, 100, 0, 1, label="💨 Flaring")

    eff_improve = gr.Slider(0, 50, 0, 1, label="⚙️ Efisiensi energi (%) — sektor energi (non-LUC)")
    target_year = gr.Slider(2024, 2035, 2030, 1, label="🎯 Target tahun proyeksi")

    run_btn = gr.Button("▶️ Jalankan Simulasi")

    # Outputs
    md = gr.Markdown()
    fig_bar = gr.Plot()
    fig_line = gr.Plot()
    table = gr.Dataframe(interactive=False)
    csv_file = gr.File(label="📥 Unduh CSV hasil")
    md_year = gr.Markdown()

    run_btn.click(
        simulate,
        inputs=[cut_luc, cut_coal, cut_oil, cut_gas, cut_cement, cut_flaring, eff_improve, target_year],
        outputs=[md, fig_bar, fig_line, table, csv_file, md_year]
    )

demo.launch(server_name="0.0.0.0", server_port=7820, share=True)


* Running on local URL:  http://0.0.0.0:7820
* Running on public URL: https://ef818de7f6de396e94.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
